# multiWavelength ptychography
 - This tutorial shows a reconstruction with a multiwavelength dataset measusred with a mouse brain sample, which is used in the publication: "L. Loetgering, X. Liu, A. C. C. De Beurs, M. Du, G. Kuijper, K. S. E. Eikema, and S. Witte, “Tailoring spatial entropy in extreme ultraviolet focused beams for multispectral ptychography,” Optica 8, 130 (2021)".
 - Download the dataset from figshare: https://figshare.com/articles/dataset/Brain_smoothBeam_poly_bin8_hdf5/19107068
 - A kind suggestion: check out general tutorials for CPM (conventional ptychographic microscopy) before diving into this one.

In [1]:
# %matplotlib notebook
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import h5py

In [2]:
# import the fracPy module
import PtyLab
from PtyLab import ExperimentalData
from PtyLab import Reconstruction
from PtyLab import Monitor
from PtyLab import Params
from PtyLab import Engines

C:\Users\User\.conda\envs\EX01\lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [3]:
from PtyLab.utils.visualisation import hsvplot, hsvmodeplot, absplot, absmodeplot

In [4]:
# Step 1: Download the preprocessed data from surfdrive (https://figshare.com/articles/dataset/Brain_smoothBeam_poly_bin8_hdf5/19107068)

# set your data folder as fileFolder 
fileFolder = Path("/Users/User/Documents/Ptychography/PtyLab.py/datasets/")
# select the hdf5 file
fileName = "Brain_smoothBeam_poly_bin8.hdf5"
# create the complete filePath
filePath = Path.joinpath(fileFolder,fileName)

In [5]:
# Step 2: Initialize the ExperimentalData class 

# Initialize the ExperimentalData using the filePath, and choose the operation Mode (default is 'CPM')
experimentalData = ExperimentalData(filePath, operationMode='CPM')
backgroundOffset = 10
experimentalData.entrancePupilDiameter = 0.2e-3
from PtyLab.utils.utils import posit
experimentalData.ptychogram = posit(experimentalData.ptychogram-backgroundOffset)
# show measured ptychogram in log scale. Need to close the window to continue
experimentalData.showPtychogram()

Found encoder with shape (890, 2)
"Unable to synchronously open object (object 'orientation' doesn't exist)"
Min max ptychogram: 0.0, 14378.9013671875
Min max ptychogram: 0.0, 4.1577558517456055
0.0 4.157756


In [6]:
# Set monitor properties
monitor = Monitor()
monitor.figureUpdateFrequency = 1
monitor.objectPlot = 'angle'  # complex abs angle
monitor.verboseLevel = 'high'  # high: plot two figures, low: plot only one figure
monitor.probeZoom = 1  # control probe plot FoV
monitor.objectZoom = 1   # control object plot FoV
monitor.objectPlotContrast = 8
monitor.probePlotContrast = 1

In [10]:
params = Params()
params.gpuSwitch = True
params.positionOrder = 'random'  # 'sequential' or 'random'
params.propagatorType = 'Fraunhofer'  # Fraunhofer Fresnel ASP scaledASP polychromeASP scaledPolychromeASP
params.probePowerCorrectionSwitch = False
params.modulusEnforcedProbeSwitch = True
params.comStabilizationSwitch = True
params.orthogonalizationSwitch = True
params.orthogonalizationFrequency = 5
params.intensityConstraint = 'standard'  # standard fluctuation exponential poission
params.couplingObjectSwitch = True
params.couplingObjectAleph = 0.5
params.OPRP = False

INFO:GPU:cupy and CUDA available, switching to GPU


In [11]:
# now, all our experimental data is loaded into experimental_data and we don't have to worry about it anymore.
# now create an object to hold everything we're eventually interested in
reconstruction = Reconstruction(experimentalData, params)
reconstruction.No = 2**10
reconstruction.nlambdaObject = 1
reconstruction.nlambdaProbe = len(reconstruction.spectralDensity)

# initialize probe and object and related params
reconstruction.initialProbe = 'circ'
reconstruction.initialObject = 'ones'
reconstruction.initializeObjectProbe()

# # customize initial probe quadratic phase
reconstruction.probe = reconstruction.probe*np.exp(1.j*2*np.pi/reconstruction.wavelength *
                                             (reconstruction.Xp**2+reconstruction.Yp**2)/(1*2e-2))

hsvmodeplot(np.squeeze(reconstruction.probe), pixelSize = reconstruction.dxp, axisUnit = 'mm')

INFO:Reconstruction:Copying attribute wavelength
INFO:Reconstruction:Copying attribute dxd
INFO:Reconstruction:Copying attribute theta
INFO:Reconstruction:Copying attribute spectralDensity
INFO:Reconstruction:Copying attribute entrancePupilDiameter
INFO:Reconstruction:Initial object set to ones
INFO:Reconstruction:Initial probe set to circ


In [12]:
# Run the reconstruction
## choose engine
# mPIE
mPIE = Engines.mPIE(reconstruction, experimentalData, params, monitor)
mPIE.numIterations = 100
mPIE.betaProbe = 0.25
mPIE.betaObject = 0.25
mPIE.reconstruct()

INFO:mPIE:Sucesfully created mPIE mPIE_engine
INFO:mPIE:Wavelength attribute: 7.090000053722179e-07
INFO:mPIE:switch to gpu


ptychogram:  97%|████████████████████████████████████████████████████████████████▊  | 861/890 [00:05<00:00, 313.78it/s]
                                                                                                                       

INFO:mPIE:Doing probe com stabilization


                                                                                                                       
iteration: 0                                                                                                           
error: 130.7                                                                                                           
estimated linear overlap: 69.4 %                                                                                       
estimated area overlap: 79.5 %                                                                                         
ptychogram:  97%|█████████████████████████████████████████████████████████████████  | 865/890 [00:02<00:00, 248.13it/s]
                                                                                                                       

INFO:mPIE:Doing probe com stabilization


...

INFO:mPIE:Doing probe com stabilization


                                                                                                                       
iteration: 98                                                                                                          
error: 23.4                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 78.4 %                                                                                         
ptychogram:  99%|██████████████████████████████████████████████████████████████████▌| 885/890 [00:07<00:00, 129.84it/s]
                                                                                                                       

INFO:mPIE:Doing probe com stabilization


                                                                                                                       
iteration: 99                                                                                                          
error: 23.4                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 78.2 %                                                                                         
mPIE: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [07:51<00:00,  4.72s/it]

INFO:mPIE:switch to cpu


In [13]:
# now save the data
reconstruction.saveResults('reconstruction.hdf5')

The reconstruction results (all) have been saved


In [14]:
import os
print(os.getcwd())

C:\Users\User\Documents\Ptychography\PtyLab.py\jupyter_tutorials
